In [138]:
import glob
import json
import numpy as np
import matplotlib.pyplot as plt

In [141]:
file_name = glob.glob('./pg_ressults/*')
all_ressults = []
for i in file_name:
    print(i)
    all_ressults.append(i)

print(f'number of models: {len(all_ressults)}')

./pg_ressults/policy_gradient_Adam_0.001_0.8358189465636665.json
./pg_ressults/policy_gradient_Adam_0.0001_0.6177143132365915.json
./pg_ressults/policy_gradient_Adam_1e-06_0.6451817808033959.json
./pg_ressults/policy_gradient_Adam_0.01_0.6283296396686722.json
./pg_ressults/policy_gradient_Adam_0.0001_0.5894498228668047.json
./pg_ressults/policy_gradient_SGD_0.01_0.5945580823833224.json
./pg_ressults/policy_gradient_Adam_0.0001_0.8960957591599381.json
./pg_ressults/policy_gradient_SGD_0.001_0.6210458973742797.json
./pg_ressults/policy_gradient_SGD_1e-05_0.6428755474970488.json
./pg_ressults/policy_gradient_Adam_0.01_0.5772772232305828.json
./pg_ressults/policy_gradient_Adam_0.0001_0.5219445085843761.json
./pg_ressults/policy_gradient_SGD_0.001_0.5905848565163345.json
./pg_ressults/policy_gradient_SGD_0.01_0.6052335928835677.json
number of models: 13


In [140]:
file_path = all_ressults[12]
'''
notes

2: mase some change in index, store utslag på value og policy. Policy lr: 1e-06, Policy clip: 3, optimizer: Adam Value lr: 0.001, Value clip: value_clip

'''

def scatter_model_used(model_info, num_models, num_trials):
    counter = 0
    used_list = []
    
    for _ in range(num_models):
        used_list.append(np.full(num_trials, -10))
    
    for key, value in model_info.items():

        for used in value['used_index']:
            used_list[counter][used] = counter

        counter += 1
        
    counter_list = range(num_trials)
    
    plt.figure(figsize=(16, 10))
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.ylim(0, int(round(num_models * 1.05)))
    for i in range(num_models):
        plt.scatter(counter_list, used_list[i])
    plt.show()
    
def plot_loss(policy_list, title):
    plt.figure(figsize=(16, 10))
    #plt.ylim(0,1)
    plt.xlabel('Step', fontsize=20)
    plt.ylabel('Loss', fontsize=20)
    #plt.tick_params(axis='both', which='major', labelsize=18)
    plt.title(title, fontsize=30)

    plt.plot(policy_list,'m' ,linewidth=3, label='Loss')
    plt.legend(loc='best', fontsize=15)
    plt.show()

#Reduce the list so graph get more general and easyer to read
def generalize_list(scores, batch_size):
    score_batch = []
    score_reduced = []
    x_akses = []
    for i in range(len(scores)):
        if i % batch_size == 0 and i != 0:
            score_reduced.append(sum(score_batch)/batch_size)
            score_batch = []
            x_akses.append(i)
        score_batch.append(scores[i])
    return score_reduced, x_akses
    
def score_graph(scores, iou_scores, means, model_info, batch_size, list_length, num_models):
    plt.figure(figsize=(16, 10))
    score_reduced = []


    model_score_avg = 0
    for key, value in model_info.items():
        model_score_avg += value['score']
    model_score_avg = model_score_avg / num_models


    # x_akses is the same in both instances
    iou_scores_reduced, _ = generalize_list(iou_scores, batch_size)
    score_reduced, x_akses = generalize_list(scores, batch_size)
    mean_reduced, _ = generalize_list(means, batch_size)
    
    
    print(f' Avg score pr model: {model_score_avg}')
    
    avg = np.array([model_score_avg for i in range(list_length)])
    avg_explore = np.array([0.519 for i in range(list_length)]) # 0.5186551321699284
    #avg_iou = np.array([0.0185 for i in range(list_length)]) #0.0185 0.13364625770486935
    avg_iou = np.array([0.101 for i in range(list_length)]) #0.10119017522614594
    
    #plt.bar(, y_akses_list)
    plt.ylim(0,1)
    plt.xlabel('time', fontsize=20)
    plt.ylabel('Percentage right', fontsize=20)
    #plt.tick_params(axis='both', which='major', labelsize=18)

    plt.plot(avg, 'r--', label='Average Lower Bound')
    plt.plot(avg_explore, 'm--', label='Average score')
    plt.plot(avg_iou, 'y--', label='Average IoU')
    plt.plot(x_akses, mean_reduced,'g' ,linewidth=3, label='Mean')
    plt.plot(x_akses, score_reduced,'b' ,linewidth=3, label='pixel score')
    plt.plot(x_akses, iou_scores_reduced,'c' ,linewidth=3, label='iou score')
    plt.legend(loc='best', fontsize=15)
    plt.show()

IndexError: list index out of range

In [ ]:
with open(file_path) as reader:
    model_info = json.load(reader)



In [ ]:
print('------------------------ MODEL INFO START ------------------------')
print(f'Policy lr: {model_info["policy_lr"]}, Policy clip: {model_info["policy_clip"]}, optimizer: {model_info["optimizer"]}')
print(f'Value lr: {model_info["value_lr"]}, Value clip: {"value_clip"}')
print(f'Reward ekspontential: {model_info["reward_exponential"]}, Reward mulitiplikation: {model_info["reward_multi"]}, batch size: {model_info["batch_size"]}, box size: {model_info["box_size"]}, gamma: {model_info["gamma"]}')


In [ ]:
plot_loss(model_info['policy_loss'], 'Policy Loss')

In [ ]:
plot_loss(model_info['value_loss'], 'Value Loss')

In [ ]:
plot_loss(model_info['reward_list'], 'Reward')

In [ ]:
scatter_model_used(model_info['model_info'], model_info['num_models'], model_info['num_trials'])

In [ ]:
score_graph(model_info['pixel_scores'], model_info['iou_scores'], model_info['mean_values'], model_info['model_info'], batch_size=model_info['batch_size'], list_length=model_info['num_trials'], num_models=model_info['num_models'])

In [ ]:
for key in model_info.keys():
    print(key)